In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

from baseline_transformer import ViT_classifier


# If available, CUDA (in this case we use CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy


# Model parameters 
size = 32
bs =  512 # Batch size
lr = 0.0001
epochs = 5 # 200 would be nice but it takes too long
patch = 4
use_amp = False

# Applying some transformation to both sets 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# Prepare CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

# 10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


c:\Users\aralmeida\Anaconda3\envs\alien_test\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Files already downloaded and verified
Files already downloaded and verified


In [2]:
# MoE-based ViT call
net = ViT_classifier(image_size = 32, 
                        patch_size = 4,
                        num_classes = 10,
                        dim=412, # Original: 512
                        depth = 6,  # Original: 6
                        heads = 8, # Original: 8 
                        mlp_dim = 512, 
                        channels = 3, 
                        dim_head=512, 
                        emb_dropout=0,)    # (input_size=1000, output_size=20, num_experts=10,hidden_size=66, k= 4, noisy_gating=True)

# Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# Adam optimizer
optimizer = optim.Adam(net.parameters(), lr=lr)

# use cosine scheduling
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

# Train and test functions 
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return train_loss/(batch_idx+1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Accuraccy 
    acc = 100.*correct/total
    if acc > best_acc:
        best_acc = acc
    
    # Save in a txt accurary and loss
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, val loss: {test_loss:.5f}, acc: {(acc):.5f}'
    print(content)

    # Create txt file and append content 
    with open('baseline_ViT.txt', 'w') as f:
        f.write(content)
        f.close()

    return test_loss, acc

list_loss = []
list_acc = []

for epoch in range(0, epochs):
    start = time.time()
    trainloss = train(epoch)
    val_loss, acc = test(epoch)
    
    scheduler.step(epoch-1) # step cosine scheduling
    
    list_loss.append(val_loss)
    list_acc.append(acc)

    print(list_loss)

num_patches 64
patch_dim 48


C:\Users\aralmeida\AppData\Local\Temp\ipykernel_33456\2238442615.py:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)



Epoch: 0
Sat Aug  3 01:21:21 2024 Epoch 0, lr: 0.0001000, val loss: 164.11519, acc: 42.52000
[164.1151888370514]

Epoch: 1


c:\Users\aralmeida\Anaconda3\envs\alien_test\lib\site-packages\torch\optim\lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Sat Aug  3 02:49:57 2024 Epoch 1, lr: 0.0000905, val loss: 137.05565, acc: 51.14000
[164.1151888370514, 137.05565345287323]

Epoch: 2
Sat Aug  3 04:18:33 2024 Epoch 2, lr: 0.0001000, val loss: 122.55442, acc: 55.99000
[164.1151888370514, 137.05565345287323, 122.55441975593567]

Epoch: 3
Sat Aug  3 05:47:45 2024 Epoch 3, lr: 0.0000905, val loss: 117.03506, acc: 58.62000
[164.1151888370514, 137.05565345287323, 122.55441975593567, 117.03506034612656]

Epoch: 4
Sat Aug  3 07:16:16 2024 Epoch 4, lr: 0.0000655, val loss: 111.35362, acc: 60.52000
[164.1151888370514, 137.05565345287323, 122.55441975593567, 117.03506034612656, 111.3536222577095]
